In [24]:
import argparse
import re
import pandas as pd

# parser = argparse.ArgumentParser()
# parser.add_argument("--infile", "-i", type=str, help="CSV for dataset to expand",
#                     required=True)
# args = parser.parse_args()

csvs=["twitter_topic_0.csv","twitter_topic_1.csv","twitter_topic_2.csv","twitter_topic_3.csv"]
for csv in csvs:    
    for topic in ["lockdowns", "masking and distancing", "vaccination","missing"]:
        df = pd.read_csv(csv)
        cols = df.columns
        for col in cols:
            if "annotation" in col:
                df[col] = (df[col].notna() & df[col].str.contains(topic))
        new_fn = re.sub(r"\.csv$", f"_{topic.replace(' ', '_')}.csv", csv)
        df.to_csv(new_fn, index=False)

In [30]:
# PART 1
# PART 1
# PART 1

# 1,2 done
# 3. renamed all missing rows with "missing" & loading csv file from dataframe
import pandas as pd
from collections import Counter
from sklearn.metrics import cohen_kappa_score
import csv

# list of labels created
labels = ["lockdowns", "masking_and_distancing", "vaccination"]

# insitialising final dictionary output 
final_labels = {"lockdowns": {"text":"label"},"masking_and_distancing": {"text":"label"},"vaccination": {"text":"label"}}

# inistilaizong final dic object
accum_labels={"text":"label"}


# iterrating given 4 files of twitter_topic
for i in range(0,4):
    for label in labels:
        # reading csv
        df = pd.read_csv('twitter_topic_'+str(i)+'_' + label + '.csv', skiprows=0)
        df.columns = ["text", "an1", "an2", "an3", "an4"]
        # using sklearn cohen_kappa_score
        kappa_an_1_2 = cohen_kappa_score(df["an1"], df["an2"])
        kappa_an_1_3 = cohen_kappa_score(df["an1"], df["an3"])
        kappa_an_1_4 = cohen_kappa_score(df["an1"], df["an4"])
        kappa_an_2_3 = cohen_kappa_score(df["an2"], df["an3"])
        kappa_an_2_4 = cohen_kappa_score(df["an2"], df["an4"])
        kappa_an_3_4 = cohen_kappa_score(df["an3"], df["an4"])

        annotators = {}
        # taking mean of the kappa score
        annotators["an1"] = (kappa_an_1_2 + kappa_an_1_3 + kappa_an_1_4) / 3
        annotators["an2"] = (kappa_an_1_2 + kappa_an_2_3 + kappa_an_2_4) / 3
        annotators["an3"] = (kappa_an_1_3 + kappa_an_2_3 + kappa_an_3_4) / 3
        annotators["an4"] = (kappa_an_1_4 + kappa_an_2_4 + kappa_an_3_4) / 3
        
# PART TWO
# PART TWO
# PART TWO

        
        print("\ntwitter topic -",i,"file label", label, ":" ,str(annotators))
        
        # 1. eliminate any labels for annotators whose average kappa score is less than 0.2 (unreliable annotators)
        delItems = []
        for key, value in annotators.items():
            if value<0.2:
                delItems.append(key)
        for j in delItems:
            del annotators[j]
            df = df.drop(j, 1)
        print("\n")
        print("after delete minimum annotators", annotators)
        print("\n")
        # logic to generate the key value pairs for csv
        for index, row in df.iterrows():
            count = Counter(row[1:]).most_common(1)
            if count == 1:
                # 3. If there are ties (the same number of annotators for different labels), use the label with higher-reliability annotators (higher kappa scores on average)
                v = row[max(annotators, key=annotators.get)]
                try:
                    values = accum_labels[row[0]]
                except KeyError:
                    accum_labels[row[0]] = []
                accum_labels[row[0]].append(label)
                final_labels[v][row[0]]= v
            else:
                # 2. assign the final label to each text as the most frequent label among the remainiing annotators
                try:
                    values = accum_labels[row[0]]
                except KeyError:
                    accum_labels[row[0]] = []
                if count[0][0] is True:
                    accum_labels[row[0]].append(label)
                    final_labels[label][row[0]]= count[0][0]
                else:
                    final_labels[label][row[0]]= count[0][0]
        print('----------------------------End of file',i,'------------------------------------')



twitter topic - 0 file label lockdowns : {'an1': 0.15310922941528948, 'an2': 0.20788424756793666, 'an3': 0.1699956795349352, 'an4': 0.20986808782839614}


after delete minimum annotators {'an2': 0.20788424756793666, 'an4': 0.20986808782839614}


----------------------------End of file 0 ------------------------------------

twitter topic - 0 file label masking_and_distancing : {'an1': 0.5110966318263074, 'an2': 0.7704815888817947, 'an3': 0.7327868362752801, 'an4': 0.7630634070755388}


after delete minimum annotators {'an1': 0.5110966318263074, 'an2': 0.7704815888817947, 'an3': 0.7327868362752801, 'an4': 0.7630634070755388}


----------------------------End of file 0 ------------------------------------

twitter topic - 0 file label vaccination : {'an1': 0.673287439979103, 'an2': 0.7621309462828351, 'an3': 0.6241408556645266, 'an4': 0.7095627314652487}


after delete minimum annotators {'an1': 0.673287439979103, 'an2': 0.7621309462828351, 'an3': 0.6241408556645266, 'an4': 0.7095627314

In [26]:
# Combining all of the text/label pairs for the SECONDARY dataset into a single CSV file, with the columns "text" and "label"
labels = ["lockdowns", "masking_and_distancing", "vaccination"]
for label in labels:
    with open('twitter_topic_'+label+'.csv', 'w') as f:
        writer = csv.writer(f)
        for row in final_labels[label].items():
            writer.writerow(row)

In [27]:
# generating overall file with key as text and values as list of annotations
with open('twitter_final_label.csv', 'w') as f:
    writer = csv.writer(f)
    for row in accum_labels.items():
        writer.writerow(row)